Variance Filtering (N=1000), 3 Label

In [2]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_validate, KFold, cross_val_predict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_file_path = r"/content/drive/MyDrive/ArrayExpress-normalized.csv"
metadata_file_path = r"/content/drive/MyDrive/E-MTAB-11349.sdrf.txt"

def process_gene_expression_data(file_path, metadata_path):
    matrix = pd.read_csv(file_path)

    matrix = matrix.set_index('gene')
    matrix = matrix.T

    processed_data = pd.DataFrame(matrix)

    features_df = pd.read_csv(metadata_path, sep='\t', index_col=0)

    return processed_data, features_df

train_data, train_features = process_gene_expression_data(train_file_path, metadata_file_path)
train_data_cleaned = train_data.drop(['Unnamed: 0', 'refseq'], errors='ignore')
train_data_cleaned.index.name = 'Source Name'


In [5]:
train_data = train_data.drop(train_data.index[:2])
train_data_cleaned

gene,ACADS,ACADVL,ACAT1,PSEN1,ADA,ADRB2,ADRB3,ADSL,AGA,ALAD,...,ALOX12-AS1,CYP21A1P,LOC100506990,LOC100506123,LINC01721,PCAT19,ANKRD20A5P,LINC00547,LINC00941,LINC00700
Source Name,,,,,,,,,,,,,,,,,,,,,
Sample 1,71.394731,202.285072,51.923441,1502.534571,155.770323,236.900699,4.326953,283.415448,122.236434,257.453728,...,493.2727,18.38955,100.6017,188.2225,3.245215,33.53389,23.79824,0.0,175.2416,55.16866
Sample 2,104.829005,308.215194,71.677952,1245.404416,215.033856,267.000371,6.271821,268.79232,120.956544,275.960115,...,422.8999,42.1108,68.09405,225.7855,8.959744,27.77521,13.43962,1.791949,116.4767,42.1108
Sample 3,173.466488,271.920441,80.87289,930.624269,257.855591,470.000418,16.408992,350.449189,134.78815,364.51404,...,307.0826,4.688283,104.3143,294.1898,5.860354,21.09728,39.85041,2.344142,80.87289,26.95763
Sample 4,103.796604,232.756022,80.992805,754.098056,247.696442,394.741631,6.290703,419.118106,61.334357,319.253192,...,392.3826,3.93169,69.19774,379.8012,7.077041,38.53056,18.87211,0.786338,99.07858,40.10323
Sample 5,83.472519,219.115362,57.861632,1089.885502,145.128357,181.173308,11.382616,315.8676,113.826162,239.983492,...,352.8611,128.0544,115.7233,518.8576,48.37612,129.9515,29.40509,36.9935,350.0154,207.7327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample 586,147.529335,278.935246,71.74924,1035.123862,216.860061,388.574534,11.286397,366.80791,112.863973,274.098219,...,336.1734,12.89874,80.61712,232.9835,4.837027,24.99131,24.99131,4.030856,91.09735,42.72708
Sample 587,94.187393,306.109028,53.515564,1343.240664,172.320117,187.304475,6.421868,278.280934,126.296732,323.234008,...,325.3746,5.351556,96.32802,180.8826,3.210934,34.24996,11.77342,8.56249,107.0311,62.07805
Sample 588,87.027568,230.201954,45.853235,2358.166354,111.357856,363.082756,10.293583,320.036862,79.541325,230.201954,...,417.358,10.29358,78.60555,126.3303,6.550462,33.68809,4.678901,2.807341,174.9909,46.78901


In [24]:
columns_to_keep = ['Characteristics[age]', 'Characteristics[sex]', 'Characteristics[disease]']
filtered_train_features = train_features[columns_to_keep]
filtered_train_features = filtered_train_features.rename(columns={
    'Characteristics[age]': 'Age',
    'Characteristics[sex]': 'Sex',
    'Characteristics[disease]': 'Disease'
})

disease_mapping = {
    'normal': 0,
    "Crohn's disease": 1,
    'ulcerative colitis': 2
}

filtered_train_features['Disease_Label'] = filtered_train_features['Disease'].map(disease_mapping)
filtered_train_features['Age'] = pd.to_numeric(filtered_train_features['Age'], errors='coerce')

filtered_train_features = filtered_train_features.drop(columns=['Disease'])
#filtered_train_features["Sex"] = filtered_train_features["Sex"].replace({"male": 0, "female": 1})

In [26]:
filtered_train_features

,Age,Sex,Disease_Label
Source Name,,,
Sample 1,34,male,1
Sample 2,49,female,0
Sample 3,27,male,0
Sample 4,9,male,0
Sample 5,34,female,0
...,...,...,...
Sample 586,25,male,1
Sample 587,44,male,0
Sample 588,36,female,1


In [27]:
common_samples = train_data_cleaned.index.intersection(filtered_train_features.index)
X = train_data_cleaned.loc[common_samples].values  # n x p
aux_vars = filtered_train_features.loc[common_samples, ['Age', 'Sex']]

aux_vars['Sex'] = (aux_vars['Sex'].str.lower() == 'female').astype(int)
X_aux = aux_vars[['Age','Sex']].values  # n x p_aux
y = filtered_train_features.loc[common_samples,'Disease_Label'].values

In [28]:
#pre-processing -- using the most common + prominant features
gene_means = X.mean(axis=0)
mean_threshold = 10.0
high_mean_idx = np.where(gene_means > mean_threshold)[0]
X_filtered = X[:, high_mean_idx]
print(f"After mean filtering: {X_filtered.shape[1]} genes remain.")


N = 1000
gene_vars = X_filtered.var(axis=0)
top_gene_idx = np.argsort(gene_vars)[-N:]
X_reduced = X_filtered[:, top_gene_idx]
print(f"After variance filtering: {X_reduced.shape[1]} genes remain.")

use_additional_feature_selection = False
if use_additional_feature_selection:
    # We have a multi-class classification problem: use logistic regression with L1 penalty
    # For a quick approximation, we might treat disease as multi-class. Some solvers (like 'saga') support L1 with multinomial.
    clf = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=1000, C=0.1, random_state=42)
    clf.fit(X_reduced, y)
    # Get the absolute coefficients sum across classes for each gene as importance
    importance = np.sum(np.abs(clf.coef_), axis=0)
    # Keep top M features by importance
    M = 500
    top_features = np.argsort(importance)[-M:]
    X_final = X_reduced[:, top_features]
    print(f"After L1 logistic regression feature selection: {X_final.shape[1]} genes remain.")
else:
    X_final = X_reduced

train_data = pd.DataFrame(X_final, index=common_samples, columns=train_data_cleaned.columns[high_mean_idx][top_gene_idx])

After mean filtering: 14137 genes remain.
After variance filtering: 1000 genes remain.


In [29]:
print(train_data.shape)
print(filtered_train_features.shape)

train_data

(590, 1000)
(590, 3)


gene,TRAPPC1,C5AR2,SRSF5,ATP5F1B,KBTBD12,PPM1A,GNA12,CKAP4,LRCH4,BPI,...,IFITM2,ALAS2,UBB,ACTB,RMRP,RPPH1,HSP90AB2P,HBA1,HBA2,HBB
Source Name,,,,,,,,,,,,,,,,,,,,,
Sample 1,1453.856345,2545.330342,4427.555075,1488.471972,991.954069,1060.103585,630.653459,572.239588,4750.994842,93.029498,...,37941.9727,5050.636366,18349.52767,95384.44267,74338.14129,113875.6781,89562.52686,496827.2808,496530.8845,452504.1335
Sample 2,1969.351732,1989.959143,3918.096052,1936.200679,772.329933,842.215936,598.510899,848.487757,3613.464756,61.822234,...,56090.68538,6522.693634,17889.92085,145541.1856,116353.9235,195915.5543,61212.07505,626927.6235,627700.8494,485705.9304
Sample 3,2367.583152,1420.549891,3050.900468,2783.66831,536.808457,665.736253,1436.958883,541.496741,2907.907822,132.444008,...,31405.63889,10021.20592,41102.18117,135807.8514,310195.5875,322998.1175,40210.23524,1330698.938,1332779.364,1615222.658
Sample 4,1443.716404,956.186899,5154.445005,1838.458035,665.241872,575.599351,542.573158,289.372351,3893.158996,107.728294,...,10722.50375,5958.868688,16690.02216,81713.87667,124977.4022,66735.71215,61475.89786,649970.4039,650682.8261,651143.6201
Sample 5,1377.296561,1008.310086,3904.23736,1529.064777,2930.075123,642.169265,702.876551,355.706756,2482.358885,149.871113,...,26015.9179,5853.510386,13435.28133,102815.378,80987.31433,93663.75458,482285.243,616130.5814,616574.5034,544536.7711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample 586,2029.939162,988.36593,4198.539775,2533.796182,637.681444,559.482835,673.95915,248.300739,2816.762284,58.044329,...,33426.27778,7690.873551,21450.60414,91070.74554,230275.5568,203162.406,36424.42859,561478.9148,560784.8014,493534.8034
Sample 587,2124.567903,1185.904905,3813.519112,2866.293624,810.225644,1086.365955,2284.044284,445.249495,2826.692106,110.242063,...,19434.71233,26088.8376,55833.85855,133186.3255,123150.0166,112637.4191,48100.8595,1129617.234,1128455.947,1176640.29
Sample 588,2247.744279,2054.973537,3434.313698,1812.606439,961.982147,1146.330866,952.624345,1034.973011,3987.359855,122.587219,...,105024.4945,9823.821581,26563.99535,101705.2818,138670.4752,249880.4775,46310.83122,904293.1636,905864.3388,1093725.041


In [30]:
filtered_train_features["Sex"] = filtered_train_features["Sex"].replace({"male": 0, "female": 1})  #makes sex an integer rather than string

all_data = pd.concat([train_data, filtered_train_features], axis=1)
all_data.columns = all_data.columns.astype(str)

all_data_train, all_data_test = train_test_split(all_data, test_size=0.2, random_state=42)

<ipython-input-30-1d314e0a4b03>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_train_features["Sex"] = filtered_train_features["Sex"].replace({"male": 0, "female": 1})  #makes sex an integer rather than string


In [31]:
print(all_data.shape)
all_data

(590, 1003)


,TRAPPC1,C5AR2,SRSF5,ATP5F1B,KBTBD12,PPM1A,GNA12,CKAP4,LRCH4,BPI,...,ACTB,RMRP,RPPH1,HSP90AB2P,HBA1,HBA2,HBB,Age,Sex,Disease_Label
Source Name,,,,,,,,,,,,,,,,,,,,,
Sample 1,1453.856345,2545.330342,4427.555075,1488.471972,991.954069,1060.103585,630.653459,572.239588,4750.994842,93.029498,...,95384.44267,74338.14129,113875.6781,89562.52686,496827.2808,496530.8845,452504.1335,34,0,1
Sample 2,1969.351732,1989.959143,3918.096052,1936.200679,772.329933,842.215936,598.510899,848.487757,3613.464756,61.822234,...,145541.1856,116353.9235,195915.5543,61212.07505,626927.6235,627700.8494,485705.9304,49,1,0
Sample 3,2367.583152,1420.549891,3050.900468,2783.66831,536.808457,665.736253,1436.958883,541.496741,2907.907822,132.444008,...,135807.8514,310195.5875,322998.1175,40210.23524,1330698.938,1332779.364,1615222.658,27,0,0
Sample 4,1443.716404,956.186899,5154.445005,1838.458035,665.241872,575.599351,542.573158,289.372351,3893.158996,107.728294,...,81713.87667,124977.4022,66735.71215,61475.89786,649970.4039,650682.8261,651143.6201,9,0,0
Sample 5,1377.296561,1008.310086,3904.23736,1529.064777,2930.075123,642.169265,702.876551,355.706756,2482.358885,149.871113,...,102815.378,80987.31433,93663.75458,482285.243,616130.5814,616574.5034,544536.7711,34,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample 586,2029.939162,988.36593,4198.539775,2533.796182,637.681444,559.482835,673.95915,248.300739,2816.762284,58.044329,...,91070.74554,230275.5568,203162.406,36424.42859,561478.9148,560784.8014,493534.8034,25,0,1
Sample 587,2124.567903,1185.904905,3813.519112,2866.293624,810.225644,1086.365955,2284.044284,445.249495,2826.692106,110.242063,...,133186.3255,123150.0166,112637.4191,48100.8595,1129617.234,1128455.947,1176640.29,44,0,0
Sample 588,2247.744279,2054.973537,3434.313698,1812.606439,961.982147,1146.330866,952.624345,1034.973011,3987.359855,122.587219,...,101705.2818,138670.4752,249880.4775,46310.83122,904293.1636,905864.3388,1093725.041,36,1,1


In [32]:
gene_data_train = all_data_train.drop(columns=['Disease_Label', "Age", "Sex"], axis=1)
gene_data_test = all_data_test.drop(columns=['Disease_Label', "Age", "Sex"], axis=1)
aux_data_train = all_data_train[["Age", "Sex"]]
aux_data_test = all_data_test[["Age", "Sex"]]
outcomes_train = all_data_train['Disease_Label']
outcomes_test = all_data_test['Disease_Label']

In [33]:
outcomes_train

,Disease_Label
Source Name,
Sample 132,1
Sample 154,2
Sample 78,1
Sample 440,1
Sample 256,1
...,...
Sample 72,0
Sample 107,0
Sample 271,2


Logistic Regression (Full Data)

In [34]:
log_reg_full_data = LogisticRegression(max_iter=5000)
log_reg_full_data.fit(pd.concat([gene_data_train, aux_data_train], axis=1), outcomes_train)

LogisticRegression(max_iter=5000)

In [35]:
predictions = log_reg_full_data.predict(pd.concat([gene_data_test, aux_data_test], axis=1))
accuracy_score(outcomes_test, predictions)

0.5423728813559322

Logistic Regression with PCA, SVD

In [36]:
#data for PCA and svd
pca = PCA(n_components=0.95).fit(gene_data_train)
gene_data_train_pca = pca.transform(gene_data_train)
gene_data_train_pca = pd.DataFrame(gene_data_train_pca)
gene_data_train_pca.columns = gene_data_train_pca.columns.astype(str)
gene_data_test_pca = pca.transform(gene_data_test)
gene_data_test_pca = pd.DataFrame(gene_data_test_pca)
gene_data_test_pca.columns = gene_data_test_pca.columns.astype(str)

svd = TruncatedSVD().fit(gene_data_train)
gene_data_train_svd = svd.transform(gene_data_train)
gene_data_train_svd = pd.DataFrame(gene_data_train_svd)
gene_data_train_svd.columns = gene_data_train_svd.columns.astype(str)
gene_data_test_svd = svd.transform(gene_data_test)
gene_data_test_svd = pd.DataFrame(gene_data_test_svd)
gene_data_test_svd.columns = gene_data_test_svd.columns.astype(str)

In [37]:
# Check for NaNs in the PCA-transformed and auxiliary datasets
print(gene_data_train_pca.isna().sum())
print(gene_data_test_pca.isna().sum())
print(aux_data_train.isna().sum())
print(aux_data_test.isna().sum())

0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
Age    0
Sex    0
dtype: int64
Age    0
Sex    0
dtype: int64


In [38]:
print("Training Sets")
print("Data: ", gene_data_train_pca.shape)
print("Auxiliary: ", aux_data_train.shape)

print()
print("Testing Sets")
print("Data: ", gene_data_test_pca.shape)
print("Auxiliary: ", aux_data_test.shape)

Training Sets
Data:  (472, 3)
Auxiliary:  (472, 2)

Testing Sets
Data:  (118, 3)
Auxiliary:  (118, 2)


In [39]:
#fixing alignment of the data
gene_data_test_pca = gene_data_test_pca.reset_index(drop=True)
aux_data_test = aux_data_test.reset_index(drop=True)

In [40]:
#Logistic Regression with PCA
log_reg_pca = LogisticRegression(max_iter=1000)
log_reg_pca.fit(np.concatenate([gene_data_train_pca, aux_data_train], axis=1), outcomes_train)

predictions_pca = log_reg_pca.predict(pd.concat([gene_data_test_pca, aux_data_test], axis=1))
accuracy_score(outcomes_test, predictions_pca)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.4576271186440678

In [41]:
print("Training Sets")
print("Data: ", gene_data_train_svd.shape)
print("Auxiliary: ", aux_data_train.shape)

print()
print("Testing Sets")
print("Data: ", gene_data_test_svd.shape)
print("Auxiliary: ", aux_data_test.shape)

Training Sets
Data:  (472, 2)
Auxiliary:  (472, 2)

Testing Sets
Data:  (118, 2)
Auxiliary:  (118, 2)


In [42]:
#Logistic Regression with SVD
log_reg_svd = LogisticRegression(max_iter=1000)
log_reg_svd.fit(np.concatenate([gene_data_train_svd, aux_data_train], axis=1), outcomes_train)

predictions_svd = log_reg_svd.predict(pd.concat([gene_data_test_svd, aux_data_test], axis=1))
accuracy_score(outcomes_test, predictions_svd)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.4745762711864407

Support Vector Machines (SVMs) + PCA/SVD

In [43]:
#PCA with Support Vector Machines
kernel = "linear"
svm_pca = SVC(kernel = kernel, max_iter=2000)     #initialize model

In [44]:
#Train the model using PCA transformed training data
svm_pca.fit(gene_data_train_pca, outcomes_train)

#Make predictions
predictions_svm_pca = svm_pca.predict(gene_data_test_pca)
#Calculate Accuracy
accuracy_pca = accuracy_score(outcomes_test, predictions_pca)

print("Accuracy for SVM with PCA:", accuracy_pca)

Accuracy for SVM with PCA: 0.4576271186440678


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [45]:
#SVMs with SVD
kernel = "linear"
svm_svd = SVC(kernel = kernel, max_iter=1000)     #initialize model

In [46]:
#Train the model using SVD transformed training data
svm_svd.fit(gene_data_train_svd, outcomes_train)

#Make predictions
predictions_svm_svd = svm_svd.predict(gene_data_test_svd)
#Calculate Accuracy
accuracy_svd = accuracy_score(outcomes_test, predictions_svd)

print("Accuracy for SVM with PCA:", accuracy_svd)

Accuracy for SVM with PCA: 0.4745762711864407


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Scaling Version of SVD+SVM and PCA+SVM

In [47]:
#Initialize Standard Scaler
scaler = StandardScaler()

# Scale the training and testing data using Standard Scaling
gene_data_train_scaled = scaler.fit_transform(gene_data_train)
gene_data_test_scaled = scaler.transform(gene_data_test)

In [48]:
#PCA + SVM + Standard Scaler

# Apply PCA to data
pca = PCA(n_components=0.95)
gene_data_train_pca_scaled = pca.fit_transform(gene_data_train_scaled)
gene_data_test_pca_scaled = pca.transform(gene_data_test_scaled)

# Initialize the SVM model
kernel='linear'
svm_pca_scaled = SVC(kernel=kernel, max_iter=1000)

# Train the SVM model on scaled and PCA-transformed data
svm_pca_scaled.fit(gene_data_train_pca_scaled, outcomes_train)

# Make predictions
predictions_pca_scaled = svm_pca_scaled.predict(gene_data_test_pca_scaled)
accuracy_pca_scaled = accuracy_score(outcomes_test, predictions_pca_scaled)

print(accuracy_pca_scaled)

0.4915254237288136


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [49]:
#SVD + SVM + Standard Scaler

#Transform data to svd
gene_data_train_svd_scaled = svd.fit_transform(gene_data_train_scaled)
gene_data_test_svd_scaled = svd.transform(gene_data_test_scaled)

# Initialize the SVM model
kernel='linear'
svm_svd_scaled = SVC(kernel=kernel, max_iter=1000)

# Train the SVM model on scaled and PCA-transformed data
svm_svd_scaled.fit(gene_data_train_svd_scaled, outcomes_train)

# Make predictions
predictions_svd_scaled = svm_svd_scaled.predict(gene_data_test_svd_scaled)
accuracy_svd_scaled = accuracy_score(outcomes_test, predictions_svd_scaled)

print(accuracy_svd_scaled)

0.3559322033898305


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
